# Smart DB Connector — Interactive Tests

This notebook runs interactive tests for `smart_db_connector.py`.
It mirrors the unit tests but shows outputs inline for visual inspection.

In [1]:
# Test connection to Neon database
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Complete connection string for Neon database
database_url = (
    "postgresql+psycopg2://neondb_owner:npg_CeS9fJg2azZD"
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech:5432/neondb"
    "?sslmode=require"
)

print("Testing connection to Neon database...")

# Test 1: Direct psycopg2 connection
try:
    conn = psycopg2.connect(
        host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
        port=5432,
        database="neondb",
        user="neondb_owner",
        password="npg_CeS9fJg2azZD",
        sslmode="require"
    )
    print("✅ Direct psycopg2 connection successful")
    
    # Test basic query
    cursor = conn.cursor()
    cursor.execute("SELECT version()")
    version = cursor.fetchone()
    print(f"✅ Database version: {version[0]}")
    
    cursor.close()
    conn.close()
except Exception as e:
    print(f"❌ Direct psycopg2 connection failed: {e}")

Testing connection to Neon database...
✅ Direct psycopg2 connection successful
✅ Database version: PostgreSQL 17.5 on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit


In [2]:
# Test 2: SQLAlchemy engine
try:
    from sqlalchemy import text
    engine = create_engine(database_url)
    
    # Test connection
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1 as test"))
        print("✅ SQLAlchemy engine connection successful")
        print(f"✅ Test query result: {result.fetchone()}")
        
except Exception as e:
    print(f"❌ SQLAlchemy connection failed: {e}")

✅ SQLAlchemy engine connection successful
✅ Test query result: (1,)


In [3]:
# Test 3: Using your db_connector class
try:
    # Use complete connection string
    db = db_connector(database_url)
    
    # Test health check
    health = db.health_check()
    print(f"✅ db_connector health check: {health}")
    
    if health['status'] == 'healthy':
        print("✅ Connection to Neon database successful!")
    else:
        print(f"❌ Health check failed: {health}")
        
except Exception as e:
    print(f"❌ db_connector failed: {e}")

❌ db_connector failed: name 'db_connector' is not defined


In [4]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd())))

import pandas as pd
from smart_db_connector import *

# Connect to Neon database with complete connection string
database_url = (
    "postgresql+psycopg2://neondb_owner:npg_CeS9fJg2azZD"
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech:5432/neondb"
    "?sslmode=require"
)

db = db_connector(database_url)
print("✅ Connected to Neon database")
db

✅ Connected to Neon database


## 1. Health Check

In [5]:
health = db.health_check()
health

{'status': 'healthy',
 'database_version': 'PostgreSQL 17.5 on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit',
 'connection_time_ms': 1934.323787689209,
 'timestamp': '2025-08-19T15:34:46.051410'}

## 2. smart_execute — create table + insert rows

In [6]:
# Cell for creating table and inserting data
db.smart_execute("CREATE TABLE IF NOT EXISTS users (id SERIAL PRIMARY KEY, name VARCHAR(100), active BOOLEAN)")
db.smart_execute("INSERT INTO users (name, active) VALUES ('Alice', true)")
db.smart_execute("INSERT INTO users (name, active) VALUES ('Bob', false)")

res = db.smart_execute("UPDATE users SET active = true WHERE name = 'Bob'", analyze_impact=True)
res

{'success': True,
 'rows_affected': 4,
 'report': SmartReport(operation='SMART_EXECUTE', success=True, execution_time_ms=923.5818386077881, data_summary={}, performance={}, memory_usage_mb=0.0, cpu_time_ms=0.0, io_operations=0, info_messages=['[15:34:48] Starting smart execute: UPDATE operation', '[15:34:49] Connection health check passed', '[15:34:49] Transaction committed automatically', '[15:34:49] Connection pool utilization: 0/5', '[15:34:49] Execute completed: 4 rows affected'], warning_messages=[], error_messages=[], recommendation_messages=[], connection_healthy=True, database_version='PostgreSQL 17.5 on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit', connection_time_ms=347.33080863952637, timestamp='2025-08-19T15:34:48.790506'),
 'sql_type': 'UPDATE',
 'impact_analysis': {'estimated_affected_rows': 0,
  'tables_involved': [],
  'risk_level': 'LOW',
  'recommendations': []}}

## 3. smart_insert — insert a DataFrame

In [7]:
# Cell for DataFrame insertion
df = pd.DataFrame([{"name": "Carol", "active": True}, {"name": "Dave", "active": False}])
ins = db.smart_insert(df, table="users_staging", if_exists="replace", create_indexes=True)
ins

{'success': True,
 'rows_inserted': 2,
 'report': SmartReport(operation='SMART_INSERT', success=True, execution_time_ms=3583.235740661621, data_summary={}, performance={}, memory_usage_mb=0.0, cpu_time_ms=0.0, io_operations=0, info_messages=['[15:34:49] Starting smart insert: 2 rows to users_staging', '[15:34:50] Connection health check passed', '[15:34:50] DataFrame validation passed: 2 rows, 2 columns', '[15:34:50] Schema and table preparation completed', '[15:34:53] Connection pool utilization: 0/5', '[15:34:53] Insert completed: 2 rows inserted'], warning_messages=[], error_messages=[], recommendation_messages=[], connection_healthy=True, database_version='PostgreSQL 17.5 on aarch64-unknown-linux-gnu, compiled by gcc (Debian 12.2.0-14+deb12u1) 12.2.0, 64-bit', connection_time_ms=339.4780158996582, timestamp='2025-08-19T15:34:49.731717'),
 'table_created': False,
 'indexes_created': []}

## 4. smart_query — fetch data as DataFrame + SmartReport

In [8]:
# Cell for querying data
out = db.smart_query("SELECT * FROM users WHERE active = true", analyze_data=True)
display(out["data"].head())
out["report"].summary()

,id,name,active
0,1,Alice,True
1,3,Alice,True
2,5,Alice,True
3,7,Alice,True
4,2,Bob,True


{'operation': 'SMART_QUERY',
 'success': True,
 'execution_time_ms': 1038.99,
 'data_summary': {'rows': 0, 'columns': 0, 'size_mb': 0.0, 'quality_score': 0},
 'performance': {'connection_time_ms': 0.0, 'memory_mb': 0.0, 'io_ops': 0},
 'status': {'info_count': 5, 'warnings': 0, 'errors': 0, 'recommendations': 0},
 'messages': {'info': ['[15:34:53] Starting smart query operation',
   '[15:34:53] Connection health check passed',
   '[15:34:53] Query automatically optimized',
   '[15:34:54] Connection pool utilization: 0/5',
   '[15:34:54] Query completed successfully: 8 rows retrieved'],
  'warnings': [],
  'errors': [],
  'recommendations': []}}

## 5. get_table_info

In [9]:
info = db.get_table_info("users")
info

{'exists': True,
 'columns': [{'name': 'id',
   'type': INTEGER(),
   'nullable': False,
   'default': "nextval('users_id_seq'::regclass)",
   'autoincrement': True,
   'comment': None},
  {'name': 'name',
   'type': VARCHAR(length=100),
   'nullable': True,
   'default': None,
   'autoincrement': False,
   'comment': None},
  {'name': 'active',
   'type': BOOLEAN(),
   'nullable': True,
   'default': None,
   'autoincrement': False,
   'comment': None}],
 'primary_key': ['id'],
 'foreign_keys': [],
 'indexes': [],
 'column_count': 3,
 'has_primary_key': True}

## 6. SmartReport message collection

In [10]:
rep = SmartReport(operation="TEST")
rep.add_info("info msg")
rep.add_warning("warn msg")
rep.add_error("error msg")
rep.add_recommendation("recommend msg")

# Show full summary with messages
summary = rep.summary()
print("=== SmartReport Summary ===")
for key, value in summary.items():
    print(f"{key}: {value}")

# Show messages using correct attribute names
print("\n=== Messages ===")
print("Info:", rep.info)
print("Warnings:", rep.warnings)
print("Errors:", rep.errors)
print("Recommendations:", rep.recommendations)

=== SmartReport Summary ===
operation: TEST
success: False
execution_time_ms: 0.0
data_summary: {'rows': 0, 'columns': 0, 'size_mb': 0.0, 'quality_score': 0}
performance: {'connection_time_ms': 0.0, 'memory_mb': 0.0, 'io_ops': 0}
status: {'info_count': 1, 'warnings': 1, 'errors': 1, 'recommendations': 1}
messages: {'info': ['[15:34:57] info msg'], 'warnings': ['[15:34:57] warn msg'], 'errors': ['[15:34:57] error msg'], 'recommendations': ['[15:34:57] recommend msg']}

=== Messages ===
Info: ['[15:34:57] info msg']
Warnings: ['[15:34:57] warn msg']
Errors: ['[15:34:57] error msg']
Recommendations: ['[15:34:57] recommend msg']


## 7. Run comprehensive demo (smoke test)

In [11]:
run_comprehensive_demo()

🎯 db_connector Demo Starting...
✅ Connected to database...
✅ Health check passed...
✅ Demo table created...
✅ Sample data inserted...
✅ Query executed, found 2 rows
🎉 Demo completed successfully!
✅ Connection closed


## 8. Close connection

In [12]:
db.close()